## Projeto - Regressão Linear 

### A Hyundai, uma das maiores comercializantes de cruzeiros, quer que você faça um modelo de predição que os diga quantos trabalhadores os futuros cruzeiros deles irão precisar.
- Utilizar o arquivo "cruise_ship_info.csv" para treinamento e testagem do modelo
- Pesquisar como converter uma coluna do tipo string em números (StringIndexer) na documentação
- Atenção aos cruzeiros que diferem em número de funcionários possíveis

In [44]:
# Importação das bibliotecas necessárias
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import corr

In [2]:
# Inicialização da SparkSession
spark = SparkSession\
    .builder\
    .appName('lr_project')\
    .getOrCreate()

spark

In [3]:
# Importação dos dados para um dataframe
dados = spark.read.format('csv').load('C:/Users/drumo/OneDrive/Documentos/Estudo/Programação/VSCode/Python/Datasets/cruise_ship_info.csv', inferSchema=True, header=True)
dados.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [4]:
# Verificação dos dados
dados.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
nomes_traduzidos = [
    'Nome Navio',
    'Linha Cruzeiro', # Empresa que utiliza o navio
    'Idade', # Idade do navio
    'Tonelagem', # Capacidade de um veículo de transporte
    'Num Passageiros', # Numero médio de passageiros por navio
    'Comprimento', 
    'Num Cabines', # Numero de cabines
    'Densidade Passageiros', 
    'Media Equipes' # Numero médio de integrantes da equipe presente no navio
    ]

In [6]:
dados = dados.toDF(*nomes_traduzidos)

In [7]:
dados.printSchema()

root
 |-- Nome Navio: string (nullable = true)
 |-- Linha Cruzeiro: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Tonelagem: double (nullable = true)
 |-- Num Passageiros: double (nullable = true)
 |-- Comprimento: double (nullable = true)
 |-- Num Cabines: double (nullable = true)
 |-- Densidade Passageiros: double (nullable = true)
 |-- Media Equipes: double (nullable = true)



In [50]:
indexado = StringIndexer(inputCol='Linha Cruzeiro', outputCol='Categoria Cruzeiro')
indexado = indexado.fit(dados).transform(dados)
indexado.head(3)

[Row(Nome Navio='Journey', Linha Cruzeiro='Azamara', Idade=6, Tonelagem=30.276999999999997, Num Passageiros=6.94, Comprimento=5.94, Num Cabines=3.55, Densidade Passageiros=42.64, Media Equipes=3.55, Categoria Cruzeiro=16.0),
 Row(Nome Navio='Quest', Linha Cruzeiro='Azamara', Idade=6, Tonelagem=30.276999999999997, Num Passageiros=6.94, Comprimento=5.94, Num Cabines=3.55, Densidade Passageiros=42.64, Media Equipes=3.55, Categoria Cruzeiro=16.0),
 Row(Nome Navio='Celebration', Linha Cruzeiro='Carnival', Idade=26, Tonelagem=47.262, Num Passageiros=14.86, Comprimento=7.22, Num Cabines=7.43, Densidade Passageiros=31.8, Media Equipes=6.7, Categoria Cruzeiro=1.0)]

In [75]:
# Vetorização das colunas necessárias: 
assembler = VectorAssembler(
    inputCols=[
        'Idade',
        'Tonelagem',
        'Densidade Passageiros',
        'Comprimento',
        'Num Cabines', 
        'Densidade Passageiros',
        'Categoria Cruzeiro'],
    outputCol='features'
) 

In [76]:
novos_dados = assembler.transform(indexado)

In [77]:
novos_dados.select('features', 'Media Equipes').show(10)

+--------------------+-------------+
|            features|Media Equipes|
+--------------------+-------------+
|[6.0,30.276999999...|         3.55|
|[6.0,30.276999999...|         3.55|
|[26.0,47.262,31.8...|          6.7|
|[11.0,110.0,36.99...|         19.1|
|[17.0,101.353,38....|         10.0|
|[22.0,70.367,34.2...|          9.2|
|[15.0,70.367,34.2...|          9.2|
|[23.0,70.367,34.2...|          9.2|
|[19.0,70.367,34.2...|          9.2|
|[6.0,110.23899999...|         11.5|
+--------------------+-------------+
only showing top 10 rows



In [78]:
dados_finais = novos_dados.select('Media Equipes', 'features')
dados_finais.show(10)

+-------------+--------------------+
|Media Equipes|            features|
+-------------+--------------------+
|         3.55|[6.0,30.276999999...|
|         3.55|[6.0,30.276999999...|
|          6.7|[26.0,47.262,31.8...|
|         19.1|[11.0,110.0,36.99...|
|         10.0|[17.0,101.353,38....|
|          9.2|[22.0,70.367,34.2...|
|          9.2|[15.0,70.367,34.2...|
|          9.2|[23.0,70.367,34.2...|
|          9.2|[19.0,70.367,34.2...|
|         11.5|[6.0,110.23899999...|
+-------------+--------------------+
only showing top 10 rows



In [79]:
dados_treino, dados_teste = dados_finais.randomSplit([0.7, 0.3])

In [80]:
dados_teste.describe().show()

+-------+------------------+
|summary|     Media Equipes|
+-------+------------------+
|  count|                42|
|   mean|  7.85452380952381|
| stddev|3.8221856870445943|
|    min|              0.88|
|    max|              19.1|
+-------+------------------+



In [81]:
dados_treino.describe().show()

+-------+------------------+
|summary|     Media Equipes|
+-------+------------------+
|  count|               116|
|   mean| 7.772327586206896|
| stddev|3.3980839505863276|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [82]:
dados_regressao = LinearRegression(labelCol='Media Equipes')

In [83]:
modelo_linear = dados_regressao.fit(dados_treino)
resultado_teste = modelo_linear.evaluate(dados_teste)

In [84]:
resultado_teste.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -0.710094333232311|
|-0.40670088179743713|
|-0.40794465946359315|
| -1.2547478575224396|
| -0.4429645990786697|
|-0.16984999418657232|
|0.045917569742218944|
|-0.22917073052891723|
|  1.1969740186396827|
| -0.4157452899645362|
|-0.12697628582615827|
|  0.1780969211162704|
| 0.17768232856088506|
| -0.5340090867905234|
| -1.1599855951927465|
|   -1.16019289147044|
| -0.0775948899881973|
|  0.7209461847576959|
|  0.2586118072523389|
|0.025469277341981567|
+--------------------+
only showing top 20 rows



C:\spark-3.2.0-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [86]:
print(f'Intercept:{modelo_linear.intercept}')
print(f'Coefficients:{modelo_linear.coefficients}')

Intercept:-1.9637639627228725
Coefficients:[0.00020729627769276637,-0.007199417201394892,0.00932556502008408,0.38182171977871227,0.6854260993461246,0.00932556502008408,0.05346433943036581]


In [87]:
print(f'''RMSE: {resultado_teste.rootMeanSquaredError}
MSE: {resultado_teste.meanSquaredError}
R²: {resultado_teste.r2}''')

RMSE: 1.516009876399715
MSE: 2.298285945341479
R²: 0.8388442034111554


In [88]:
unlabeled_data = dados_teste.select('features')
predictions = modelo_linear.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[27.0,5.35,32.04,...| 1.590094333232311|
|[21.0,10.0,48.08,...| 2.006700881797437|
|[27.0,10.0,48.08,...|2.0079446594635932|
|[23.0,14.745,47.8...|3.0547478575224396|
|[19.0,16.8,56.76,...|  2.54296459907867|
|[14.0,33.0,67.35,...|3.4098499941865725|
|[15.0,30.27699999...| 3.954082430257781|
|[12.0,50.0,71.43,...| 4.679170730528917|
|[10.0,46.0,65.71,...| 3.273025981360317|
|[18.0,51.004,54.2...| 5.865745289964536|
|[20.0,55.451,43.8...| 5.696976285826159|
|[17.0,55.451,43.8...|5.7019030788837295|
|[19.0,55.451,43.8...| 5.702317671439115|
|[27.0,53.872,36.0...|6.6540090867905235|
|[20.0,50.76,29.04...| 7.299985595192746|
|[21.0,50.76,29.04...|  7.30019289147044|
|[5.0,122.0,34.57,...|6.7775948899881975|
|[21.0,47.225,34.5...| 5.979053815242304|
|[12.0,42.0,28.38,...| 6.541388192747661|
|[9.0,59.058,34.74...| 7.374530722658019|
+--------------------+------------

In [89]:
dados.select(corr('Media Equipes', 'Num Passageiros')).show()

+------------------------------------+
|corr(Media Equipes, Num Passageiros)|
+------------------------------------+
|                  0.9152341306065384|
+------------------------------------+



estou falando do outro lado do mundo!


In [90]:
dados.select(corr('Media Equipes', 'Num Cabines')).show()

+--------------------------------+
|corr(Media Equipes, Num Cabines)|
+--------------------------------+
|              0.9508226063578497|
+--------------------------------+

